In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Strawberry___healthy', 'Strawberry___Leaf_scorch']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(7499, 128, 128, 3)

In [8]:
y.shape

(7499, 4)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(5999, 128, 128, 3)

In [11]:
X_test.shape

(1500, 128, 128, 3)

In [12]:
y_train.shape

(5999, 4)

In [13]:
y_test.shape

(1500, 4)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

4

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 13:46:49.471828: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


188/188 [==============================] - 91s 478ms/step - loss: 0.4496 - accuracy: 0.8296 - val_loss: 0.5343 - val_accuracy: 0.8227
Epoch 2/10
188/188 [==============================] - 91s 483ms/step - loss: 0.1829 - accuracy: 0.9347 - val_loss: 0.0834 - val_accuracy: 0.9693
Epoch 3/10
188/188 [==============================] - 93s 493ms/step - loss: 0.0935 - accuracy: 0.9665 - val_loss: 0.1866 - val_accuracy: 0.9340
Epoch 4/10
188/188 [==============================] - 81s 432ms/step - loss: 0.0491 - accuracy: 0.9847 - val_loss: 0.7679 - val_accuracy: 0.7740
Epoch 5/10
188/188 [==============================] - 87s 463ms/step - loss: 0.0557 - accuracy: 0.9810 - val_loss: 0.3446 - val_accuracy: 0.9067
Epoch 6/10
188/188 [==============================] - 115s 611ms/step - loss: 0.0740 - accuracy: 0.9767 - val_loss: 0.1539 - val_accuracy: 0.9520
Epoch 7/10
188/188 [==============================] - 136s 726ms/step - loss: 0.0512 - accuracy: 0.9828 - val_loss: 0.1430 - val_accuracy: 0

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 96.67%


In [19]:
cnn_scores

[0.08621338754892349, 0.9666666388511658]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

47/47 [==============================] - 7s 135ms/step
Accuracy: 96.67%
Precision: 96.87%
Recall: 96.67%
F1 score: 96.69%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-pepper+strawberry.csv', index=False)

+------+----------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                             | True Labels                   | Predicted Labels              |
|------+----------------------------------------------------------------------+-------------------------------+-------------------------------|
|    0 | 04cb725a-632d-44e0-9886-c84e2bbdd3da___RS_HL 2094_90deg.JPG          | Strawberry___healthy          | Strawberry___healthy          |
|    1 | b1ead01a-3f1c-4f9c-a91e-0ddaa45112a4___RS_L.Scorch 0783_flipLR.JPG   | Strawberry___Leaf_scorch      | Strawberry___Leaf_scorch      |
|    2 | 9029703b-db1b-4227-bd29-ab05cc53b7ba___NREC_B.Spot 1802_180deg.JPG   | Pepper,_bell___Bacterial_spot | Pepper,_bell___Bacterial_spot |
|    3 | c47ccf21-5249-4a95-8d33-0d640e3529f7___RS_L.Scorch 0158.JPG          | Strawberry___Leaf_scorch      | Strawberry___Leaf_scorch

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-pepper+strawberry.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------+--------------------------+-------------------------------+
|      | Filename                                                             | True Labels              | Predicted Labels              |
|------+----------------------------------------------------------------------+--------------------------+-------------------------------|
|   10 | 8fcd1d14-b91c-4d1b-9c8b-db4fdd59ce5a___RS_L.Scorch 0118.JPG          | Strawberry___Leaf_scorch | Pepper,_bell___Bacterial_spot |
|   42 | ad4d5ac8-0cea-4d31-8108-3d6c6977bbc3___JR_HL 8138.JPG                | Pepper,_bell___healthy   | Pepper,_bell___Bacterial_spot |
|  119 | ca33c3f3-52d8-4401-8a6a-962a9b13f11d___RS_L.Scorch 9912_flipLR.JPG   | Strawberry___Leaf_scorch | Pepper,_bell___Bacterial_spot |
|  137 | eaff2763-f80a-495c-b4b8-15ad0fdbe3bb___JR_HL 8100.JPG                | Pepper,_bell___healthy   | Pepper,_bell___Bacterial_spot |
|  146 |

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

188/188 [==============================] - 27s 145ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 69.48013091087341 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

47/47 [==============================] - 5s 113ms/step
Random Forest Classifier Accuracy: 99.07%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-pepper+strawberry.csv', index=False)

+------+----------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                             | True Labels                   | Predicted Labels              |
|------+----------------------------------------------------------------------+-------------------------------+-------------------------------|
|    0 | 04cb725a-632d-44e0-9886-c84e2bbdd3da___RS_HL 2094_90deg.JPG          | Strawberry___healthy          | Strawberry___healthy          |
|    1 | b1ead01a-3f1c-4f9c-a91e-0ddaa45112a4___RS_L.Scorch 0783_flipLR.JPG   | Strawberry___Leaf_scorch      | Strawberry___Leaf_scorch      |
|    2 | 9029703b-db1b-4227-bd29-ab05cc53b7ba___NREC_B.Spot 1802_180deg.JPG   | Pepper,_bell___Bacterial_spot | Pepper,_bell___Bacterial_spot |
|    3 | c47ccf21-5249-4a95-8d33-0d640e3529f7___RS_L.Scorch 0158.JPG          | Strawberry___Leaf_scorch      | Strawberry___Leaf_scorch

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-pepper+strawberry.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                             | True Labels                   | Predicted Labels              |
|------+----------------------------------------------------------------------+-------------------------------+-------------------------------|
|    4 | 7614ff40-fd01-4c7c-929f-f74814c5de74___RS_L.Scorch 1448.JPG          | Strawberry___Leaf_scorch      | Strawberry___healthy          |
|  119 | ca33c3f3-52d8-4401-8a6a-962a9b13f11d___RS_L.Scorch 9912_flipLR.JPG   | Strawberry___Leaf_scorch      | Pepper,_bell___Bacterial_spot |
|  194 | 1be321d5-d2cc-4945-b2f3-a36c41160838___JR_HL 7596_new30degFlipLR.JPG | Pepper,_bell___healthy        | Strawberry___healthy          |
|  213 | 0e57b44f-bb06-43ec-8688-5a7985b461e7___JR_B.Spot 8970.JPG            | Pepper,_bell___Bacterial_spot | Peppe